<font size = "5"> 1.1 </font>
<body>
<font size = "2" color = "red">->Put raw csv in same path as ipynb for smooth running of code </font><br><br>

<font size = "3">This part of the code finds the number of houses in to whole dataset,<br>
outputs <b><i>clean_df</i></b> csv for further processing,<br>
marks out the time period of all malfunction meter</font><br>
</body>

In [7]:
import numpy as np
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
import timeit
from datetime import timedelta
from pathlib import Path
# start_time = timeit.default_timer() * 1000
currentPath = str(Path().resolve())
# Put raw csv in same 
path = currentPath + "./dataport-export_gas_oct2015-mar2016.csv"

# Read in raw data and find 
df = pd.read_csv(path)
data_id = df['dataid'].nunique()
print("Number of houses =", data_id)

Number of houses = 157


In [8]:
# Sort data into order by id, if same id then by time
df.sort_values(["dataid", "localminute"], ascending=[True, True], inplace = True)
df.reset_index(drop = True, inplace = True)

# Cast localminute into proper time format for further processing
df.localminute = df.localminute.str.slice(0,19)
df.localminute = pd.to_datetime(df.localminute, infer_datetime_format = True, format = "%Y/%m/%d %I:%M:%S %p")
print(df.localminute[0])
print("Length of df:", len(df))

# df = df[df.dataid == 2449]

2015-10-01 00:14:44
Length of df: 1584823


In [9]:
# df = df[620670:620746]
# df[0:10]

In [10]:
malfunction = pd.DataFrame(columns = ["localminute", "dataid", "meter_value"])
bad_array = []
to_drop = []
good_array = []
prev_bad_time = -1
good_data = df.meter_value[0]
prev_good = True
prev_good_index = 0

def IsNextID(row):
    return (row.dataid != df.dataid[row.Index - 1])

def markGood(row):
    global prev_good
    global good_data
    global good_array
    global prev_good_index
    prev_good_index = row.Index
    prev_good = True
    good_data = row.meter_value
    good_array.append(row.Index)

def dropBad(row):
    global prev_good 
    global prev_bad_time
    global bad_array
    global to_drop
    to_drop.append(row.Index)
    if (prev_good == True):
        bad_array.append([row.dataid, row.localminute, row.localminute])
        prev_bad_time = row.localminute
    else:
        bad_array[-1][2] = row.localminute
    prev_good = False

def markBad(row):
    global bad_array
    global prev_good
    global prev_bad_time
    if (prev_good == True):
        bad_array.append([row.dataid, row.localminute, row.localminute])
        prev_bad_time = row.localminute
    else:
        bad_array[-1][2] = row.localminute

def checkIncreasing(row):
    return (row.meter_value > df.meter_value[row.Index - 1])

def checkDiffOverTwo(row):
    return ((row.meter_value - df.meter_value[row.Index - 1]) > 2)

def checkConstant(row):
    global prev_bad_time  
    if (prev_bad_time == -1):
        prev_bad_time = row.localminute
        return False
    if (row.meter_value == df.meter_value[row.Index - 1]):
        return ((row.localminute - prev_bad_time) >= pd.to_timedelta("12:00:00"))
    else:
        return False

def checkDrasticJump(row):
    return ((row.meter_value - df.meter_value[prev_good_index]) > 168)

def checkHighRate(row):
    # True if > 7/hr
    norm = (row.localminute - df.localminute[row.Index - 1]) / timedelta(hours = 1) * 7
    change = (row.meter_value - df.meter_value[row.Index - 1]) 
    if (change < 0):
        return False
    return (norm < change)

def checkPrevGood(row):
    diff = row.meter_value - good_data
    if (diff >= 2 and diff < 168):
        return True
    else:
        return False
def isTwo(row):
    return ((row.meter_value - df.meter_value[prev_good_index]) == 2)
    
for row in df.itertuples():
    if (row.Index == 0):
        markGood(row)
#         print("first data", row.localminute, row.dataid, row.meter_value, "good")
        continue
    if(IsNextID(row)):
        markGood(row)
        continue
    else:
        if (isTwo(row)):
            markGood(row)
#             print("=2       .", row.localminute, row.dataid, row.meter_value, "good")
        else:
            if (checkIncreasing(row)):
                if (not checkDrasticJump(row)):
                    if(not checkHighRate(row)):
                        markGood(row)
#                         print("!=2 ^ !b!h", row.localminute, row.dataid, row.meter_value, "good")
                    else:
                        markBad(row)
#                         print("!=2 ^ !bhb", row.localminute, row.dataid, row.meter_value, "mbad")
                else:
                    dropBad(row)
#                     print("!=2 ^ bh d", row.localminute, row.dataid, row.meter_value, "bad")
            else:
                if (checkPrevGood(row)):
                    if(not checkHighRate(row)):
                        markGood(row)
#                         print("!=2 !^ p g", row.localminute, row.dataid, row.meter_value, "good")
                    else:
                        markBad(row)
#                         print("!=2 !^ !pg", row.localminute, row.dataid, row.meter_value, "mbad")
                else:
                    if (checkConstant(row)):
                        markGood(row)
#                         print("!=2 !^ 12", row.localminute, row.dataid, row.meter_value, "good")
                    else:
                        dropBad(row)
#                         print("!=2 !^ !12", row.localminute, row.dataid, row.meter_value, "bad")

print("Length of bad_array:", len(bad_array))
print("Length of to_drop:", len(to_drop))

Length of bad_array: 213000
Length of to_drop: 1147869


In [11]:
# # Types of malfunction
# # 1: data reported back when change in gas use is < 2 cubic foot
# # 2: data reported back new meter_value is smaller than old meter_value
# # 3: data reported back is >2, but time > 15s, threshold value for reporting malfunction will be 7 cubic foot/hr 
# # (US household average daily usage = 168 cubic foot) https://www.aga.org/sites/default/files/aga_2961_2015_aga_playbook_final_0.pdf
# # If report between 2 reading is > 168, the extra high reading will be treated as malfunction and dropped away
# # 4: However, continuous report of same reading from same id over 12hrs,
# # the first reading after 12hrs will be treated as good reading,
# # as the gas company will probably want to know whether is a meter malfunctioning
# # even though its reading did not change for a period of time

# # Progress bar cuz I always feel it is not working
# # f = FloatProgress(min=0, max=(len(df['dataid']) - 1))
# # display(f)

# # Create empty df and array for more efficient removal of item in df
# malfunction = pd.DataFrame(columns = ["localminute", "dataid", "meter_value"])
# bad_array = []
# to_drop = []

# prev_good = True
# _id = None
# _12hr = None
# # Sort malfunction with time period label
# # Assumption: the first data point is always correct as the 2nd pt is wrt to it, 3rd wrt to 2nd....etc
# # Append malfunction period and data to respective array
# for row in df.itertuples():
#     if(_id is None or _id != row.dataid):
#         prev_good = True
#         _id = row.dataid
#     # for estimating time take to process actual data
#     if (row.Index == 620670):
#         continue
#     #1 and #2
#     if (((row.meter_value <= df.meter_value[row.Index-1])
#          and (row.dataid == df.dataid[row.Index-1])) 
#         or ((row.localminute != df.localminute[row.Index-1]) 
#             and (row.dataid == df.dataid[row.Index-1]) 
#             and ((row.meter_value - df.meter_value[row.Index-1]) < 2))):
#         if(prev_good == True):
#             bad_array.append([_id, row.localminute, row.localminute])
#             to_drop.append(row.Index)
#             #4, update 12hr pt
#             _12hr = row.localminute
#         else:
#             #4  
#             if ((row.localminute - _12hr) >= pd.to_timedelta("12:00:00")):
#                 prev_good = True
#                 continue
#             else:                
#                 if ((df.meter_value[row.Index - 1] - row.meter_value) < 168):
#                     to_drop.append(row.Index)
#                     bad_array[-1][2] = row.localminute
                    
#         prev_good = False
#     else:
#         #3
#         if ((row.meter_value - df.meter_value[row.Index - 1]) < 168):
#             if(((row.meter_value - df.meter_value[row.Index-1]) > 2) 
#                and ((((row.localminute - df.localminute[row.Index-1]) / timedelta(hours = 1)) * 7) 
#                     > (row.meter_value - df.meter_value[row.Index - 1]))):
#                 bad_array[-1][2] = row.localminute
#             prev_good = True
#         else:
#             to_drop.append(row.Index)
#             prev_good = False
#             bad_array[-1][2] = row.localminute
            
# print("Length of bad_array:", len(bad_array))
# print("Length of to_drop:", len(to_drop))

In [12]:
#Remove malfunction data to produce clean df
df.drop(index = to_drop, inplace = True)
df.reset_index(drop = True, inplace = True)

# # Cast to int64
# malfunction['dataid'] = malfunction['dataid'].astype(np.int64)
# malfunction['meter_value'] = malfunction['meter_value'].astype(np.int64)

# # Merge 2 df to compare diff, if one of the label value is different,
# # value in _merge label will be different hence being able to differeniate the difference
# df = pd.merge(df, malfunction, on=['localminute', 'dataid', 'meter_value'], how='outer', indicator=True)\
# .query("_merge != 'both'")\
# .drop(['_merge'], axis=1)\
# .reset_index(drop=True)

# Convert bad_array into df and transform it for shape to be correct
malfunction = pd.DataFrame(bad_array, columns = ['dataid', 'start_time', 'end_time'])
malfunction.T

# Save part1 result into csv for easier access in part2
df.to_csv('./clean_df.csv', index = False)
malfunction.to_csv('./malfunction.csv', index = False)

# pd.set_option('display.max_rows', 10000)

# elapsed = timeit.default_timer() * 1000 - start_time
# print("total: %ds" %(elapsed/1000)) if ((elapsed > 5000) == True) else print("total: %dms" %elapsed)
print("length of df:", len(df))
print("length of malfunction:", len(malfunction))

length of df: 436954
length of malfunction: 213000
